In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('dataset.csv')

In [3]:
import plotly.graph_objects as go

def plot_signal_by_index(dataset, index):
    """
    Grafica la señal de pupilometría correspondiente al índice dado en el dataset.
    
    Parameters:
    - dataset: DataFrame que contiene las señales con las columnas `sample_1` a `sample_2770`.
    - index: Índice de la fila en el dataset que se desea graficar.
    """
    # Verificar que el índice esté dentro del rango del dataset
    if index < 0 or index >= len(dataset):
        raise IndexError("Índice fuera de rango del dataset.")
    
    # Extraer las 2770 muestras de la fila correspondiente al índice
    samples = dataset.loc[index, [f'sample_{i+1}' for i in range(2770)]]

    # Extraer la información de la fila correspondiente al índice
    patient_info = dataset.loc[index, ['id', 'eye', 'stimuli', 'color', 'glaucoma']]
    samples = dataset.loc[index, [f'sample_{i+1}' for i in range(2770)]].values

    # Mostrar información del paciente
    print("Información del paciente:")
    print(f"ID: {patient_info['id']}")
    print(f"Ojo: {patient_info['eye']}")
    print(f"Estímulo: {patient_info['stimuli']}")
    print(f"Color: {patient_info['color']}")
    print(f"Glaucoma: {patient_info['glaucoma']}")
    
    
    # Crear el gráfico con plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(1, 2771)),  # El eje x representa los números de las muestras (1 a 2770)
        y=samples,
        mode='lines',
        name=f'Señal de índice {index}',
        hovertemplate='Muestra: %{x}<br>Valor: %{y:.2f}'
    ))

    # Añadir título y etiquetas de los ejes
    fig.update_layout(
        title=f'Señal del índice {index}',
        xaxis_title='Número de muestra',
        yaxis_title='Valor de la señal',
        template='plotly_white'
    )
    
    # Mostrar el gráfico
    fig.show()

## Saturar

In [4]:
import pandas as pd
import numpy as np

def saturate_values(dataset, max_value):
    """
    Genera un nuevo dataset en el que todos los valores superiores al `max_value` son saturados.
    
    Parameters:
    - dataset: El DataFrame original que contiene las señales de las muestras.
    - max_value: El valor límite a partir del cual los valores serán saturados.
    
    Returns:
    - El dataset con los valores saturados.
    """
    # Copiar el dataset original para no modificar el original
    saturated_dataset = dataset.copy()

    # Seleccionar las columnas que contienen las muestras
    sample_columns = [col for col in dataset.columns if col.startswith('sample_')]
    
    # Aplicar la saturación (clipping) a las columnas de las muestras
    saturated_dataset[sample_columns] = saturated_dataset[sample_columns].clip(upper=max_value)
    
    return saturated_dataset

In [5]:
dataset_saturated = saturate_values(dataset, max_value=80)

In [15]:
plot_signal_by_index(dataset, 31)

Información del paciente:
ID: C002
Ojo: right
Estímulo: ramp
Color: red
Glaucoma: no


In [7]:
plot_signal_by_index(dataset_saturated, 31)

Información del paciente:
ID: C002
Ojo: right
Estímulo: ramp
Color: red
Glaucoma: no


## Padding

In [8]:
plot_signal_by_index(dataset, 20)

Información del paciente:
ID: C002
Ojo: left
Estímulo: flash
Color: white
Glaucoma: no


In [9]:
plot_signal_by_index(dataset, 201)

Información del paciente:
ID: C013
Ojo: right
Estímulo: flash
Color: blue
Glaucoma: no


In [10]:
from tqdm import tqdm

def fill_nan_with_padding(dataset, num_padding=300):
    """
    Rellena los valores NaN en señales de tipo 'flash' copiando las últimas 300 muestras válidas.
    
    Parameters:
    - dataset: El DataFrame original que contiene las señales.
    - num_padding: El número de muestras válidas que se usarán para rellenar los NaN.
    
    Returns:
    - El dataset con los valores NaN de las señales 'flash' rellenados.
    """
    # Copiar el dataset original para no modificar el original
    filled_dataset = dataset.copy()

    # Identificar las señales de clase 'flash'
    flash_signals = filled_dataset[filled_dataset['stimuli'] == 'flash']
    
    # Seleccionar las columnas que contienen las muestras
    sample_columns = [col for col in filled_dataset.columns if col.startswith('sample_')]

    # Barra de progreso con tqdm
    for index, row in tqdm(flash_signals.iterrows(), total=flash_signals.shape[0], desc="Rellenando NaN para señales 'flash'"):
        # Convertir las muestras a float para evitar problemas con np.isnan
        signal_samples = pd.to_numeric(row[sample_columns], errors='coerce').values
        
        # Encontrar los valores no NaN (válidos) en la señal
        valid_samples = signal_samples[~np.isnan(signal_samples)]
        
        # Asegurarse de que existan al menos 300 muestras válidas
        if len(valid_samples) >= num_padding:
            # Tomar las últimas `num_padding` muestras válidas
            padding_samples = valid_samples[-num_padding:]
            
            # Rellenar los NaN con las últimas `num_padding` muestras
            nan_indices = np.isnan(signal_samples)
            signal_samples[nan_indices] = np.tile(padding_samples, (nan_indices.sum() // num_padding + 1))[:nan_indices.sum()]
            
            # Actualizar la fila en el dataset original
            filled_dataset.loc[index, sample_columns] = signal_samples
        else:
            print(f"Advertencia: la señal en el índice {index} no tiene suficientes muestras válidas para el padding.")
    
    return filled_dataset


In [11]:
dataset_filled = fill_nan_with_padding(dataset_saturated, num_padding=300)

Rellenando NaN para señales 'flash':  15%|█▍        | 49/328 [00:23<02:11,  2.12it/s]


KeyboardInterrupt: 

In [32]:
plot_signal_by_index(dataset_filled, 20)

Información del paciente:
ID: C002
Ojo: left
Estímulo: flash
Color: white
Glaucoma: no


In [33]:
plot_signal_by_index(dataset_filled, 201)

Información del paciente:
ID: C013
Ojo: right
Estímulo: flash
Color: blue
Glaucoma: no


In [34]:
# Verificar si el DataFrame tiene valores nulos
def check_nulls(dataset):
    nulls = dataset.isnull().sum()
    print("Valores nulos por columna:")
    print(nulls[nulls > 0])  # Mostrar solo las columnas con valores nulos
    return nulls[nulls > 0]

In [35]:
nulls = check_nulls(dataset)

Valores nulos por columna:
sample_1785      4
sample_1786      4
sample_1787      4
sample_1788      4
sample_1789      4
              ... 
sample_2766    328
sample_2767    328
sample_2768    328
sample_2769    328
sample_2770    328
Length: 986, dtype: int64


In [36]:
nulls = check_nulls(dataset_filled)

Valores nulos por columna:
Series([], dtype: int64)


In [38]:
total_nulls = dataset.isnull().sum().sum()
print(f"Total de valores nulos en el dataset: {total_nulls}")

Total de valores nulos en el dataset: 312672


In [37]:
total_nulls = dataset_filled.isnull().sum().sum()
print(f"Total de valores nulos en el dataset: {total_nulls}")

Total de valores nulos en el dataset: 0


In [39]:
def save_dataset_to_csv(dataset, file_name="dataset_sat_fill.csv"):
    """
    Guarda el dataset en un archivo CSV.
    
    Parameters:
    - dataset: El DataFrame que se desea guardar.
    - file_name: El nombre del archivo CSV (default: "dataset.csv").
    
    Returns:
    - Ruta del archivo CSV generado.
    """
    file_path = f'{file_name}'  # Ruta para guardar el archivo CSV en el entorno del servidor
    dataset.to_csv(file_path, index=False)  # Guardar el DataFrame en formato CSV
    return file_path

In [40]:
file_path = save_dataset_to_csv(dataset_filled)